In [4]:
##intermidiate model training
import os
import subprocess
file_name="MyFile150"
ranking_method="randomForests"
cmd=["java","-jar","RankLib-2.13.jar","-train","uploadedFiles/"+file_name+".txt","-tvs","0.75","-ranker",'8',"-metric2t","NDCG@100","-metric2T","ERR@10","-save","trainedModels/"+'intermidiate_'+ranking_method+".txt","-gmax","9","-norm","zscore"]
if ranking_method=='listNet':
    cmd.append("-epoch")
    cmd.append("6000")
output=[]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE)
for line in proc.stdout.readlines():
    x=str(line)
    x=x[2:len(x)-3]
    x=x.replace('\\t',' ')
    print(x)
    output.append(x)
index=1

\r
Discard orig. features\r
Training data: uploadedFiles/MyFile150.txt\r
Train-Validation split: 0.75\r
Feature vector representation: Dense.\r
Ranking method: Random Forests\r
Feature description file: Unspecified. All features will be used.\r
Train metric: NDCG@100\r
Test metric: ERR@10\r
Highest relevance label (to compute ERR): 9\r
Feature normalization: zscore\r
Model file: trainedModels/intermidiate_randomForests.txt\r
\r
[+] Random Forests's Parameters:\r
No. of bags: 300\r
Sub-sampling: 1.0\r
Feature-sampling: 0.3\r
No. of trees: 1\r
No. of leaves: 100\r
No. of threshold candidates: 256\r
Learning rate: 0.1\r
\r
\rReading feature file [uploadedFiles/MyFile150.txt]: 0... \rReading feature file [uploadedFiles/MyFile150.txt]... [Done.]            \r
(11 ranked lists, 1650 entries read)\r
Initializing... [Done]\r
------------------------------------\r
Training starts...\r
------------------------------------\r
bag       | NDCG@100- | NDCG@100-OO | \r
-------------------------------

In [74]:
##Step1 ranking
import os
import subprocess
file_name="MyFile8"
ranking_method="randomForests"

#Start#generating ranklist for input for given input
cmd=["java","-jar","RankLib-2.13.jar","-rank","uploadedFiles/"+file_name+".txt","-load","trainedModels/"+ranking_method+".txt","-norm","zscore","-indri","myNewRankedLists.txt"]
output=[]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE)
for line in proc.stdout.readlines():
    x=str(line)
    x=x[2:len(x)-3]
    x=x.replace('\\t',' ')
#     print(x)
    output.append(x)
#End#
    
#Start#
score_data={}
label_data = {}
query={}
query_name = {}
file1 = open("uploadedFiles/"+file_name+".txt","r")
for x in file1:
    curr_row=x.split(" ")
    qid=int(curr_row[1].split(":")[1])
    date=float(curr_row[14].split(":")[1])
    index2=int(curr_row[26].split("_")[0])-1
    if qid not in score_data:
        label_data[qid] = {}
        score_data[qid] = []
        query[qid]=curr_row[26].split("_")[2]
        rem = x.split("=")[1]
        i = rem.find("of")
        rem = rem[i+3:]
        rem = rem[:len(rem)-1]
        query_name[qid] = rem
    label_data[qid][index2+1] = int(curr_row[0])
    score_data[qid].append([0,date,index2+1])
file1.close()
f = open("myNewRankedLists.txt","r")
f1 = f.readlines()
ans = {}
for x in f1:
    y = x.split()
    value = y[len(y)-2]
    q_id = y[0]
    z = x.split('_')
    i = z[0]
    i = i.split()
    # print(i)
    i = i[4]
    for v in range(len(score_data[int(q_id)])):
        # print(score_data[int(q_id)][v][2])
        # print(i)
        if score_data[int(q_id)][v][2] == int(i):
            score_data[int(q_id)][v][0] = float(value)
            break
for id in score_data.keys():
    score_data[id].sort(key = lambda sub: (-sub[0], sub[1]))
inputInfo=output[6]
inputInfo=inputInfo[1:len(inputInfo)-1]
modelInfo=output[4]
#End#

#Start#
file1 = open("myNewRankedLists.txt","w")
for qid in score_data.keys():
    cnt = 1
    for doc in score_data[qid]:
        s=str(qid)+" Q0 docid = "+str(doc[2])+"_of_"+query_name[qid]+" "+str(cnt)+" "+str(doc[0])+" indri\n"
        cnt = cnt+1
#         print(s)
        file1.write(s)
file1.close()


In [75]:
#Step2 ranking(top-n)
#Start#get top-n tweets id
n=150
file = open("myNewRankedLists.txt","r")
lines = file.readlines()
counter=0
top_n={}
for line in lines:
    line_data=line.split(" ")
#     print(line_data[4])
    qid=line_data[0]
    if qid not in top_n:
        top_n[qid]=[]
        counter=0
    if counter<n:
        top_n[qid].append(line_data[4])
    counter=counter+1
#End#

In [76]:
#get details of top-n tweets and make new file for testing into second phase
file1 = open("uploadedFiles/"+file_name+".txt","r")
tweets={}
for x in file1:
    curr_row=x.split(" ")
    l=len(curr_row)
    doc_id=curr_row[l-1]
    doc_id=doc_id[:len(doc_id)-1]
    tweets[doc_id]=x
    
file1 = open("uploadedFiles/intermidiate_"+file_name+".txt","w")
remove_feature_no=13
for key in top_n.keys():
    for doc in top_n[key]:
        doc_info=tweets[doc].split(" ")
        left_part=doc_info[:14]
        right_part=doc_info[15:]
        for i in range(0,len(right_part)):
            parti=right_part[i].split(":")
            if len(parti)==2:
                parti[0]=str(int(parti[0])-1)
            right_part[i]=":".join(parti)
#         print(right_part)
#         print(left_part)
        file1.write(" ".join(left_part)+" "+" ".join(right_part))
file1.close()

In [77]:
#second phase ranking 
import os
import subprocess
intermidiate_file_name="intermidiate_"+file_name
intermidiate_ranking_method="intermidiate_"+ranking_method
cmd=["java","-jar","RankLib-2.13.jar","-rank","uploadedFiles/"+intermidiate_file_name+".txt","-load","trainedModels/"+intermidiate_ranking_method+".txt","-norm","zscore","-indri","intermidiateRankedLists.txt"]
output=[]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE)
for line in proc.stdout.readlines():
    x=str(line)
    x=x[2:len(x)-3]
    x=x.replace('\\t',' ')
    print(x)
    output.append(x)

\r
Discard orig. features\r
Model file: trainedModels/intermidiate_randomForests.txt\r
Feature normalization: zscore\r
Model:  Random Forests\r
\rReading feature file [uploadedFiles/intermidiate_MyFile8.txt]: 0... \rReading feature file [uploadedFiles/intermidiate_MyFile8.txt]... [Done.]            \r
(11 ranked lists, 1650 entries read)\r


In [78]:
file = open("myNewRankedLists.txt","r")
lines = file.readlines()
rankList={}
for line in lines:
    qid=int(line.split(" ")[0])
    if qid not in rankList:
        rankList[qid]=[]
    rankList[qid].append(line)
file.close()
file = open("intermidiateRankedLists.txt","r")
lines = file.readlines()
rankList2={}
for line in lines:
    qid=int(line.split(" ")[0])
    if qid not in rankList2:
        rankList2[qid]=[]
    rankList2[qid].append(line)
file.close()

for key in rankList2.keys():
    for i in range(0, len(rankList2[key])):
        rankList[key][i]=rankList2[key][i]
file=open("myNewRankedLists.txt","w")
for key in rankList.keys():
    for i in range(0, len(rankList[key])):
        file.write(rankList[key][i])
file.close()


In [30]:
import numpy as np
def ndcg(y_true, y_score, k=1500):
        y_true_sorted = sorted(y_true, reverse=True)
        ideal_dcg = 0
        for i in range(min(k,len(y_score))):
            ideal_dcg += (2 ** y_true_sorted[i] - 1.) / np.log2(i + 2)
        dcg = 0
        argsort_indices = np.argsort(y_score)[::-1]
        for i in range(min(k,len(y_score))):
            dcg += (2 ** y_true[argsort_indices[i]] - 1.) / np.log2(i + 2)
        ndcg = dcg / ideal_dcg
        return ndcg

In [20]:
file = open("myNewRankedLists.txt","r")
lines = file.readlines()
label_map = {}
for line in lines:
    line_data=line.split(" ")
    z = line.split('_')
    i = z[0]
    i = i.split()
    docid = i[4]
    qid=line_data[0]
    value = line_data[len(line_data)-2]
    if qid not in label_map:
        label_map[qid] = []
    label_map[qid].append([100000-int(docid),docid])

In [21]:
file_name="MyFile8"
label_data = {}
file1 = open("uploadedFiles/"+file_name+".txt","r")
for x in file1:
    curr_row=x.split(" ")
    qid=int(curr_row[1].split(":")[1])
    index2=int(curr_row[26].split("_")[0])-1
    if qid not in label_data:
        label_data[qid] = {}
    label_data[qid][index2+1] = int(curr_row[0])
file1.close()


In [31]:
#NDCG calculation
tot_ndcg = 0
for id in label_map:
    y_score = []
    y_true = []
    for doc in label_map[id]:
#         print(id)
#         print(doc)
        y_score.append(int(doc[0]))
        y_true.append(label_data[int(id)][int(doc[1])])
    print(ndcg(y_true,y_score))
    tot_ndcg+=ndcg(y_true,y_score)
tot_ndcg/=len(label_data)
print(tot_ndcg)


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [26]:
label_data[1][33]

9

In [47]:
score_data={}
label_data = {}
query={}
file1 = open("uploadedFiles/"+file_name+".txt","r")
for x in file1:
    curr_row=x.split(" ")
    qid=int(curr_row[1].split(":")[1])
    date=float(curr_row[14].split(":")[1])
    index2=int(curr_row[26].split("_")[0])-1
    if qid not in score_data:
        label_data[qid] = {}
        score_data[qid] = []
        query[qid]=curr_row[26].split("_")[2]
    label_data[qid][index2+1] = int(curr_row[0])
    score_data[qid].append([0,date,index2+1])
file1.close()

In [ ]:
f = open("myNewRankedLists.txt","r")
f1 = f.readlines()
ans = {}
for x in f1:
    y = x.split()
    value = y[len(y)-2]
    q_id = y[0]
    z = x.split('_')
    i = z[0]
    i = i.split()
    # print(i)
    i = i[4]
    for v in range(len(score_data[int(q_id)])):
        # print(score_data[int(q_id)][v][2])
        # print(i)
        if score_data[int(q_id)][v][2] == int(i):
            score_data[int(q_id)][v][0] = float(value)
            break
for id in score_data.keys():
    score_data[id].sort(key = lambda sub: (-sub[0], sub[1]))